<a href="https://colab.research.google.com/github/Tuchulcha/First-HF-RL-tutorial/blob/main/HF_RL_tut_p1u1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies


In [ ]:
!pip install gym[box2d]
!pip install stable-baselines3[extra]
!pip install huggingface_sb3
!pip install pyglet
!pip install ale-py==0.7.4 # To overcome an issue with gym (https://github.com/DLR-RM/stable-baselines3/issues/875)
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 448 kB 2.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 2.7 MB/s 
     |████████████████████████████████| 1.5 MB 52.3 MB/s 
     |████████████████████████████████| 1.1 MB 38.8 MB/s 
     |████████████████████████████████| 1.6 MB 42.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616825 sha256=e2993bfc0e26dcae4e19a1020042fc958d02d5fbf6b13a5f6bd19ee3b5aec9fb
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=d705eea8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 101 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 28.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 3.2 MB/s 


Import packages

In [ ]:
import gym

from huggingface_sb3 import load_from_hub, package_to_hub, push_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv #Klippt och klistrat från slutet av tutorial

from huggingface_sb3 import package_to_hub #Klippt och klistrat från slutet av tutorial

Model, träning och utvärdering

In [ ]:
notebook_login() 
!git config --global credential.helper store
#token: (vem bryr sig)

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
# Create the environment
env = make_vec_env('LunarLander-v2', n_envs=16)
#env = gym.make('LunarLander-v2') "# Create environment" vet inte varför dem gör det två gånger


#model = PPO('MlpPolicy', env, verbose=1) "# Instantiate the agent" Två gånger igen. Amatörer?

model = PPO( # We added some parameters to accelerate the training. En gång utan parametrar(det finns default som den använder?) och sen denna...
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

model_name = "ppo-LunarLander-v2" #Vet inte om det ska vara tonothehero/ eller bara det efter? Fick ju error tidigare på "repo_id"

model.learn(total_timesteps=500000)

model_name = "ppo-LunarLander-v2" # Save the model
model.save(model_name)

#Evaluate the agent:
eval_env = gym.make("LunarLander-v2")
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
#print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 93.4     |
|    ep_rew_mean     | -199     |
| time/              |          |
|    fps             | 4064     |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 16384    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 85.8         |
|    ep_rew_mean          | -145         |
| time/                   |              |
|    fps                  | 2906         |
|    iterations           | 2            |
|    time_elapsed         | 11           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0075911935 |
|    clip_fraction        | 0.0557       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | -0.000548    

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Publicera på HF hub token

Publicera Kod

In [ ]:
#import gym
#from stable_baselines3 import PPO
#from stable_baselines3.common.vec_env import DummyVecEnv
#from stable_baselines3.common.env_util import make_vec_env
#from huggingface_sb3 import package_to_hub
############^^^^####Du hittar ovanstående i början av notebooken. Vafan gör dem här? 


# PLACE the variables you've just defined two cells above
# Define the name of the environment
env_id = "LunarLander-v2"

# TODO: Define the model architecture we used
model_architecture = "PPO"

## Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
## CHANGE WITH YOUR REPO ID
repo_id = "TonoTheHero/ppo-LunarLander-v2"

## Define the commit message
#commit_message = "Upload PPO LunarLander-v2 tutorial agent"

# Create the evaluation env
eval_env = DummyVecEnv([lambda: gym.make(env_id)])

# PLACE the package_to_hub function you've just filled here
#push_to_hub(
#    repo_id=repo_id,
#    filename="ppo-LunarLander-v2.zip",
#    commit_message="Added LunarLander-v2 model trained with PPO",
#)
package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model 
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, 
               eval_env=eval_env, 
               repo_id=repo_id, 
               commit_message="Test to avoid error")

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


NoSuchDisplayException: ignored